In [12]:
from typing import Dict, Optional, List
from pydantic import BaseModel
import uuid
from datetime import datetime
import json
from typing import Callable
from together import Together
import re
customers = []


class Customer(BaseModel):
    id: str
    name: str
    email: str
    product_choice: str
    conversation_history: List[Dict]


class Session(BaseModel):
    session_id: str = str(uuid.uuid4())
    context: Dict = {}
    current_node_id: str = "welcome"
    conversation_history: List[str] = []

class Node(BaseModel):
    id: str
    prompt_template: str
    required_fields: List[str] = []
    list_of_next_possible_nodes: List[str] = []
    custom_handler_function: Optional[Callable] = None
    def get_processed_prompt_template(self, context: Dict) -> str:
        return self.prompt_template.format(**context)
    
async def get_product_details_using_rag(question: str, history: List[str], context: Dict) -> str:
    #  TODO: Implement RAG
    return "Product A is a cutting-edge solution designed to streamline business operations and enhance productivity. With robust features including support for up to 500 concurrent users, advanced analytics capabilities, and seamless integration options, it stands out in the market. The platform offers real-time collaboration tools, automated workflow management, and customizable dashboards that adapt to your business needs. Security is paramount, with enterprise-grade encryption and role-based access control. Regular updates ensure you stay ahead with the latest features, while our 24/7 technical support team ensures smooth operations. Perfect for both small businesses and large enterprises looking to scale efficiently."

def update_scheduled_demo_details(name: str, email: str, product_choice: Optional[str] = None,date: Optional[str] = None) -> str:
    print("------------------------------ Updating scheduled demo details -----------------------------")
    print(f"Demo scheduled successfully for {name} with email {email} for product {product_choice} on {date}")
    print("--------------------------------------------------------------------------------")
    return f"Demo scheduled successfully for {name} with email {email} for product {product_choice} on {date}"

async def schedule_demo(question: str, history: List[str], context: Dict) -> str:
    # Define the function for scheduling demo
    schedule_demo_tool = {
        "name": "update_scheduled_demo_details",
        "description": "Schedule a demo with the customer",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Customer's name",
                },
                "email": {
                    "type": "string",
                    "description": "Customer's email",
                },
                "product_choice": {
                    "type": "string",
                    "description": "Product selected by customer",
                },
                "date": {
                    "type": "string",
                    "description": "Demo date (one day after today)",
                }
            },
            "required": ["name", "email"]
        }
    }

    # Create the tool prompt
    tool_prompt = f"""
    You have access to the following function:

    Use the function '{schedule_demo_tool["name"]}' to '{schedule_demo_tool["description"]}':
    {json.dumps(schedule_demo_tool)}

    Schedule a demo for tomorrow using the context provided. Format the date as YYYY-MM-DD.

    If you choose to call a function ONLY reply in the following format with no prefix or suffix:
    <function=example_function_name>{{"example_name": "example_value"}}</function>
    """

    # Initialize Together AI client
    together_client = Together()  # Replace with your API key

    # Prepare messages for the LLM
    messages = [
        {
            "role": "system",
            "content": tool_prompt,
        },
        {
            "role": "user",
            "content": f"Schedule a demo for customer with context: {json.dumps(context)}"
        }
    ]
    print("------------------------------ Messages sent to LLM -----------------------------")
    print(messages)
    print("--------------------------------------------------------------------------------")

    # Get response from LLM
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=messages,
        temperature=0,
    )

   
    print("------------------------------ Response from LLM -----------------------------")
    print(response.choices[0].message.tool_calls[0].function)
    print("--------------------------------------------------------------------------------")
    
    parsed_response = response.choices[0].message.tool_calls[0].function
    
    print("------------------------------ Parsed function calling response -----------------------------")
    print(parsed_response)
    print("--------------------------------------------------------------------------------")
    
    if parsed_response and parsed_response.name == "update_scheduled_demo_details":
        arguments = json.loads(parsed_response.arguments)
        # Call the actual function with parsed arguments
        print("------------------------------ Calling function -----------------------------")
        print("calling function", parsed_response.name)
        print("with arguments", arguments)
        print("--------------------------------------------------------------------------------")
        result = update_scheduled_demo_details(**arguments)
        return result
    
    return "Failed to schedule demo"

def _initialize_nodes() -> Dict[str, Node]:
        return {
            "welcome": Node(
                id="welcome",
                description="Welcome message",
                prompt_template="Hello! I'm your sales assistant. May I know your name?",
                required_fields=[],
                list_of_next_possible_nodes=["collect_email", "get_products", "question_and_answer_node_for_product_details"],
            ),
            "collect_name": Node(
                id="collect_name",
                description="Collect name from user",
                prompt_template="Nice to meet you, {name}! What's your email address?",
                required_fields=[],
                list_of_next_possible_nodes=["collect_email", "get_products", "question_and_answer_node_for_product_details"],
            ),
            "collect_email": Node(
                id="collect_email",
                description="Collect email from user",
                prompt_template="Nice to meet you, {name}! What's your email address?",
                required_fields=["name"],
                list_of_next_possible_nodes=["get_products"],
            ),
            "get_products": Node(
                id="get_products",
                description="Get product details from user",
                prompt_template="What product are you interested in today? We offer: \n- Product A\n- Product B\n- Product C",
                required_fields=["name", "email"],
                list_of_next_possible_nodes=["question_and_answer_node_for_product_details", "schedule_demo"],
            ),
            "question_and_answer_node_for_product_details": Node(
                id="question_and_answer_node_for_product_details",
                description="Answers any questions about the product, using RAG",
                prompt_template="",
                custom_handler_function=get_product_details_using_rag,
                required_fields=["name", "email"],
                list_of_next_possible_nodes=["schedule_demo", "end_conversation", "question_and_answer_node_for_product_details"],
               
            ),
            "schedule_demo": Node(
                id="schedule_demo",
                description="Schedule a demo with the user, if we have the name and email",
                prompt_template="Success Response: I've scheduled a demo for you. I'll send you a confirmation email shortly. \n Error Response: I'm sorry, I couldn't find your name or email in the conversation history. Please provide your name and email so I can schedule the demo.",
                required_fields=["name", "email"],
                custom_handler_function=schedule_demo,
                list_of_next_possible_nodes=["end_conversation", "question_and_answer_node_for_product_details", "collect_name", "collect_email"],
            ),
            "end_conversation": Node(
                id="end_conversation",
                description="End the conversation",
                prompt_template="Thank you for using our service. Have a great day!",
                required_fields=[],
                list_of_next_possible_nodes=[],
            ),
        }


async def _save_customer(self, session: Session):
    customer = Customer(
        id=session.session_id,
        name=session.context.get("name"),
        email=session.context.get("email"),
        product_choice=session.context.get("product_choice"),
        conversation_history=[{
            "timestamp": datetime.utcnow().isoformat(),
            "context": session.context
        }]
    )
    customers.append(customer)


def _should_save_customer(self, session: Session) -> bool:
    required_fields = ["name", "email", "product_choice"]
    return all(field in session.context for field in required_fields)

nodes = _initialize_nodes()




In [13]:
from typing import Dict
from together import Together
from pydantic import BaseModel, Field
from typing import List
import json

class InputAnalysis(BaseModel):
    next_node_id: str = Field(description="The next node id from the possible next nodes list")
    user_inputs : Dict = Field(description="Any identified user inputs from the user input eg: name, email, product_choice")
    confidence: float = Field(description="Confidence score between 0 and 1")
    suggested_response: str = Field(description="Optional suggested response", default=None)

class TogetherAIService:
    def __init__(self):
        self.client = Together()
        self.model = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

    async def analyze_intent(self, current_message: str, history: List[str], context: Dict, list_of_next_possible_nodes: List[str]) -> Dict:
        prompt = self._create_intent_prompt(current_message, history, context, list_of_next_possible_nodes, nodes)
        print("------------------------------ Prompt sent to AI -----------------------------")
        print(prompt)
        print("--------------------------------------------------------------------------------")
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a sales assistant. Only respond in JSON format."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            response_format={
                "type": "json_object",
                "schema": InputAnalysis.model_json_schema()
            }
        )
        response = self._parse_response(response)
        print("------------------------------ Parsed AI Response -----------------------------")
        print(response)
        print("--------------------------------------------------------------------------------")
        return response

    def _create_intent_prompt(self, current_message: str, history: List[str], context: Dict, list_of_next_possible_nodes: List[str], nodes: Dict[str, Node]) -> str:
        list_of_nodes = [nodes[node_id] for node_id in list_of_next_possible_nodes]
        prompt = f"""Analyze the following user input and determine the next node. if the required fields are not met, add follow up questions to the user.
        
        user details: {context}
        User Input: {current_message}
        
        History:
        {history}
        
        Possible next nodes:
        {list_of_nodes}
        
        """
        return prompt
    def _parse_response(self, response) -> Dict:
        try:
            # Parse the JSON string into a Python dictionary
            if isinstance(response.choices[0].message.content, str):
                return json.loads(response.choices[0].message.content)
            return response.choices[0].message.content
        except Exception as e:
            print("error", e)
            return {
                "intent": "unknown",
                "entities": {},
                "confidence": 0.0,
                    "suggested_response": None
                }



In [17]:
#  start conversation
session = Session()
together_service = TogetherAIService()
response = None

print(session)
while True:
    print("------------------------------ Current Node -----------------------------")
    print(session.current_node_id)
    current_node = nodes[session.current_node_id]
    suggested_response = response.get("suggested_response", None) if response else current_node.get_processed_prompt_template(session.context)
    message_to_user = suggested_response
    print("Suggested response", suggested_response)
    if current_node.custom_handler_function:
        message_to_user = await current_node.custom_handler_function(user_input, session.conversation_history, session.context)
    
    session.conversation_history.append(f'AI: {message_to_user} \n')
    print("Message sent to user:", message_to_user)
    user_input = input(f"{message_to_user} \nYou: ")
    if user_input == "exit" or user_input == "quit" or user_input == "q":
        break
    print(user_input)
    session.conversation_history.append(f'User: {user_input} \n')
    response = await together_service.analyze_intent(user_input, session.conversation_history, session.context, current_node.list_of_next_possible_nodes)
    session.current_node_id = response["next_node_id"]
    session.context = {**session.context, **response["user_inputs"]}
    print("Updated context", session.context)
    print("Updated current node", session.current_node_id)
    print("--------------------------------------------------------------------------------")
    

#  analyze intent



session_id='2670e1c5-0d45-4ad4-8b27-ca15fe837286' context={} current_node_id='welcome' conversation_history=[]
------------------------------ Current Node -----------------------------
welcome
Suggested response Hello! I'm your sales assistant. May I know your name?
Message sent to user: Hello! I'm your sales assistant. May I know your name?


Hello! I'm your sales assistant. May I know your name? 
You:  i am vineeth


i am vineeth
------------------------------ Prompt sent to AI -----------------------------
Analyze the following user input and determine the next node. if the required fields are not met, add follow up questions to the user.
        
        user details: {}
        User Input: i am vineeth
        
        History:
        ["AI: Hello! I'm your sales assistant. May I know your name? \n", 'User: i am vineeth \n']
        
        Possible next nodes:
        [Node(id='collect_email', prompt_template="Nice to meet you, {name}! What's your email address?", required_fields=['name'], list_of_next_possible_nodes=['get_products'], custom_handler_function=None), Node(id='get_products', prompt_template='What product are you interested in today? We offer: \n- Product A\n- Product B\n- Product C', required_fields=['name', 'email'], list_of_next_possible_nodes=['question_and_answer_node_for_product_details', 'schedule_demo'], custom_handler_function=None), Node(id='question_and_answer_node_for_

Nice to meet you, vineeth! What's your email address? 
You:  vineethtngl@gmail.com


vineethtngl@gmail.com
------------------------------ Prompt sent to AI -----------------------------
Analyze the following user input and determine the next node. if the required fields are not met, add follow up questions to the user.
        
        user details: {'name': 'vineeth'}
        User Input: vineethtngl@gmail.com
        
        History:
        ["AI: Hello! I'm your sales assistant. May I know your name? \n", 'User: i am vineeth \n', "AI: Nice to meet you, vineeth! What's your email address? \n", 'User: vineethtngl@gmail.com \n']
        
        Possible next nodes:
        [Node(id='get_products', prompt_template='What product are you interested in today? We offer: \n- Product A\n- Product B\n- Product C', required_fields=['name', 'email'], list_of_next_possible_nodes=['question_and_answer_node_for_product_details', 'schedule_demo'], custom_handler_function=None)]
        
        
--------------------------------------------------------------------------------
------

...

Product A is a cutting-edge solution designed to streamline business operations and enhance productivity. With robust features including support for up to 500 concurrent users, advanced analytics capabilities, and seamless integration options, it stands out in the market. The platform offers real-time collaboration tools, automated workflow management, and customizable dashboards that adapt to your business needs. Security is paramount, with enterprise-grade encryption and role-based access control. Regular updates ensure you stay ahead with the latest features, while our 24/7 technical support team ensures smooth operations. Perfect for both small businesses and large enterprises looking to scale efficiently. 
You:  schedule a demo for the same 


schedule a demo for the same 
------------------------------ Prompt sent to AI -----------------------------
Analyze the following user input and determine the next node. if the required fields are not met, add follow up questions to the user.
        
        user details: {'name': 'vineeth', 'email': 'vineethtngl@gmail.com', 'product': 'Product A'}
        User Input: schedule a demo for the same 
        
        History:
        ["AI: Hello! I'm your sales assistant. May I know your name? \n", 'User: i am vineeth \n', "AI: Nice to meet you, vineeth! What's your email address? \n", 'User: vineethtngl@gmail.com \n', 'AI: What product are you interested in today? We offer: \\\\- Product A\\\\- Product B\\\\- Product C \n', 'User: Product A \n', 'AI: Product A is a cutting-edge solution designed to streamline business operations and enhance productivity. With robust features including support for up to 500 concurrent users, advanced analytics capabilities, and seamless integration opti

Demo scheduled successfully for vineeth with email vineethtngl@gmail.com for product Product A on 2024-11-22 
You:  Thank you so much


Thank you so much
------------------------------ Prompt sent to AI -----------------------------
Analyze the following user input and determine the next node. if the required fields are not met, add follow up questions to the user.
        
        user details: {'name': 'vineeth', 'email': 'vineethtngl@gmail.com', 'product': 'Product A'}
        User Input: Thank you so much
        
        History:
        ["AI: Hello! I'm your sales assistant. May I know your name? \n", 'User: i am vineeth \n', "AI: Nice to meet you, vineeth! What's your email address? \n", 'User: vineethtngl@gmail.com \n', 'AI: What product are you interested in today? We offer: \\\\- Product A\\\\- Product B\\\\- Product C \n', 'User: Product A \n', 'AI: Product A is a cutting-edge solution designed to streamline business operations and enhance productivity. With robust features including support for up to 500 concurrent users, advanced analytics capabilities, and seamless integration options, it stands out in th

Thank you for using our service. Have a great day! 
You:  q


In [18]:
session

Session(session_id='2670e1c5-0d45-4ad4-8b27-ca15fe837286', context={'name': 'vineeth', 'email': 'vineethtngl@gmail.com', 'product': 'Product A'}, current_node_id='end_conversation', conversation_history=["AI: Hello! I'm your sales assistant. May I know your name? \n", 'User: i am vineeth \n', "AI: Nice to meet you, vineeth! What's your email address? \n", 'User: vineethtngl@gmail.com \n', 'AI: What product are you interested in today? We offer: \\\\- Product A\\\\- Product B\\\\- Product C \n', 'User: Product A \n', 'AI: Product A is a cutting-edge solution designed to streamline business operations and enhance productivity. With robust features including support for up to 500 concurrent users, advanced analytics capabilities, and seamless integration options, it stands out in the market. The platform offers real-time collaboration tools, automated workflow management, and customizable dashboards that adapt to your business needs. Security is paramount, with enterprise-grade encryption

In [19]:
from pprint import pprint

def format_session(session):
    """
    Format and print a Session object in a clean, readable way.
    
    Args:
        session: Session object containing conversation details
    """
    print("""
┌─────────────────────────────────────────────┐
│              Session Details                 │
└─────────────────────────────────────────────┘
""")
    
    print(f"Session ID: {session.session_id}\n")
    
    print("Context:")
    print(f"├── Name: {session.context.get('name')}")
    print(f"├── Email: {session.context.get('email')}")
    print(f"└── Product: {session.context.get('product') or session.context.get('product_choice')}\n")
    
    print(f"Current Node: {session.current_node_id}\n")
    
    print("┌─────────────────────────────────────────────┐")
    print("│           Conversation History               │")
    print("└─────────────────────────────────────────────┘")
    
    for message in session.conversation_history:
        # Clean up the message and handle escaped characters
        cleaned_message = message.strip().replace('\\n', '\n')
        
        # Format AI and User messages differently
        if message.startswith('AI:'):
            print(f"\n🤖 {cleaned_message[4:]}")  # Remove 'AI: ' prefix
        elif message.startswith('User:'):
            print(f"👤 {cleaned_message[6:]}")  # Remove 'User: ' prefix
        else:
            print(cleaned_message)
        
        print("─" * 50)  # Separator line

    # Print raw data if needed
    print("\nRaw Session Data:")
    pprint(vars(session), indent=2, width=100)

# Usage
if __name__ == "__main__":
    format_session(session)


┌─────────────────────────────────────────────┐
│              Session Details                 │
└─────────────────────────────────────────────┘

Session ID: 2670e1c5-0d45-4ad4-8b27-ca15fe837286

Context:
├── Name: vineeth
├── Email: vineethtngl@gmail.com
└── Product: Product A

Current Node: end_conversation

┌─────────────────────────────────────────────┐
│           Conversation History               │
└─────────────────────────────────────────────┘

🤖 Hello! I'm your sales assistant. May I know your name?
──────────────────────────────────────────────────
👤 i am vineeth
──────────────────────────────────────────────────

🤖 Nice to meet you, vineeth! What's your email address?
──────────────────────────────────────────────────
👤 vineethtngl@gmail.com
──────────────────────────────────────────────────

🤖 What product are you interested in today? We offer: \\- Product A\\- Product B\\- Product C
──────────────────────────────────────────────────
👤 Product A
──────────────────────────

In [100]:
messages = [{'role': 'system', 'content': '\n    You have access to the following function:\n\n    Use the function \'update_scheduled_demo_details\' to \'Schedule a demo with the customer\':\n    {"name": "update_scheduled_demo_details", "description": "Schedule a demo with the customer", "parameters": {"type": "object", "properties": {"name": {"type": "string", "description": "Customer\'s name"}, "email": {"type": "string", "description": "Customer\'s email"}, "product_choice": {"type": "string", "description": "Product selected by customer"}, "date": {"type": "string", "description": "Demo date (one day after today)"}}, "required": ["name", "email"]}}\n\n    Schedule a demo for tomorrow using the context provided. Format the date as YYYY-MM-DD.\n\n    If you choose to call a function ONLY reply in the following format with no prefix or suffix:\n    <function=example_function_name>{"example_name": "example_value"}</function>\n    '}, {'role': 'user', 'content': 'Schedule a demo for customer with context: {"name": "vineeth", "email": "vineetht@gmail.com", "product_interest": "Product A"}'}]
def call_function(messages):
    together_client = Together()
    response = together_client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=messages,
        temperature=0.7,
    )

    return response

response_fn = call_function(messages)
print(response_fn)


id='8e5ff65fddba7f8d-MAA' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1732184586 model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=15256571913147204000, finish_reason=<FinishReason.ToolCalls: 'tool_calls'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, tool_calls=[ToolCalls(id='call_fu4hkou77akffobj6sj33egp', type='function', function=FunctionCall(name='update_scheduled_demo_details', arguments='{"name":"vineeth","email":"vineetht@gmail.com","product_choice":"Product A","date":"2024-11-22"}'), index=0)]))] prompt=[] usage=UsageData(prompt_tokens=274, completion_tokens=48, total_tokens=322)
